In [44]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import os

# 設定 Chrome 選項
chrome_options = Options()
chrome_options.add_argument("window-size=1920,1080")  # 避免 RWD 影響
chrome_options.add_argument("--disable-gpu")

# 啟動 WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

try:
    # 1. 開啟國泰世華銀行官網
    driver.get("https://www.cathaybk.com.tw/cathaybk/")

    
## 使用Chrome App到國泰世華銀行官網(https://www.cathaybk.com.tw/cathaybk/)並將畫面截圖。
    screenshot_path = "cathaybk_screenshot.png"
    driver.save_screenshot(screenshot_path)
    print("『已截圖國泰世華頁面並儲存:cathaybk_screenshot.png』")
    
    time.sleep(1)

    
    # 2. 測試是否找到「產品介紹」
    try:
        product_intro = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), '產品介紹')]"))
        )
        print(" 找到『產品介紹』，開始點擊...")
        product_intro.click()
        time.sleep(1)
    except Exception as e:
        print(" 找不到『產品介紹』，錯誤：", e)
        
        
        
        
## 點選左上角選單，進入 個人金融 > 產品介紹 > 信用卡列表，需計算有幾個項目並將畫面截圖。
    try:
        credit_card_category = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'cubre-o-menuLinkList__item') and contains(., '信用卡')]"))
        )

        # **找到該區塊內的所有 `<a>` 連結**
        credit_card_items = credit_card_category.find_elements(By.CSS_SELECTOR, "a.cubre-a-menuLink")
        total_credit_cards = len(credit_card_items)

        print(f" 信用卡分類下的項目數量: {total_credit_cards} 個")

        # 4️⃣ 截圖「信用卡」區塊
        credit_card_screenshot_path = "credit_card_list.png"
        credit_card_category.screenshot(credit_card_screenshot_path)
        print(f"信用卡分類已截圖: {credit_card_screenshot_path}")

    except Exception as e:
        print("❌ 無法獲取信用卡分類列表，錯誤：", e)


        
        
        
    # 3. 測試是否找到「卡片介紹」
    try:
        card_intro = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), '卡片介紹')]"))
        )
        print(" 找到『卡片介紹』，開始點擊...")
        card_intro.click()
        time.sleep(1)  # 等待頁面載入
    except Exception as e:
        print(" 找不到『卡片介紹』，錯誤：", e)
        
        
##  產品介紹 > 信用卡 > 卡片介紹 > 計算頁面上所有(停發)信用卡數量並截圖
    # 4. 找到「停發卡區塊」 (`blockname06`)
    discontinued_section = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//section[@data-anchor-block='blockname06']"))
    )
    print(" 找到『停發卡區塊』")

    # 5. 創建存放截圖的資料夾
    if not os.path.exists("discontinued_cards"):
        os.mkdir("discontinued_cards")

    # 6. 記錄已截圖的卡片，避免重複
    seen_cards = set()
    total_cards = 0

    while True:
        # 7. 重新獲取目前顯示的卡片名稱（確保每次翻頁後都能獲取最新的）
        visible_cards = discontinued_section.find_elements(By.CSS_SELECTOR, "div.cubre-m-compareCard__title")

        for card in visible_cards:
            card_name = card.text.strip()
            if card_name and card_name not in seen_cards:
                seen_cards.add(card_name)
                total_cards += 1

                # **8. 只截取『停發卡區塊』，避免整個畫面**
                screenshot_path = f"discontinued_cards/{total_cards}_{card_name}.png"
                discontinued_section.screenshot(screenshot_path)  # 截圖 session 內的內容
                print(f" 截圖成功: {screenshot_path}")

        # **9. 重新找到「下一頁」按鈕**
        try:
            next_button = discontinued_section.find_element(By.CSS_SELECTOR, "div.cubre-o-slide__next.swiper-next")

            # **10. 停止條件：如果下一頁按鈕無法點擊，則停止**
            if next_button.get_attribute("aria-disabled") == "true":
                print("已到最後一張卡片，停止翻頁")
                break

            # **11. 使用 JavaScript 點擊下一頁**
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(2)  # 等待動畫

        except Exception as e:
            print("無法點擊『下一頁』，可能已到最後一張，錯誤：", e)
            break

    print(f"停發卡總數: {total_cards} 張，截圖已完成！")

except Exception as e:
    print(" 發生錯誤:", e)

finally:
    driver.quit()


『已截圖國泰世華頁面並儲存:cathaybk_screenshot.png』
 找到『產品介紹』，開始點擊...
 信用卡分類下的項目數量: 9 個
信用卡分類已截圖: credit_card_list.png
 找到『卡片介紹』，開始點擊...
 找到『停發卡區塊』
 截圖成功: discontinued_cards/1_Costco聯名卡.png
 截圖成功: discontinued_cards/2_eTag聯名卡【已停發】.png
 截圖成功: discontinued_cards/3_(COMBO)悠遊聯名卡(停發).png
 截圖成功: discontinued_cards/4_COMBO財富卡(停發).png
 截圖成功: discontinued_cards/5_NFC行動信用卡(停發).png
 截圖成功: discontinued_cards/6_Taiwan Money 卡 (停發).png
 截圖成功: discontinued_cards/7_國民旅遊卡(停發).png
 截圖成功: discontinued_cards/8_聯合報優秀卡(停發).png
 截圖成功: discontinued_cards/9_W@card(停發).png
 截圖成功: discontinued_cards/10_遠東SOGO(悠遊)聯名卡.png
 截圖成功: discontinued_cards/11_太平洋百貨聯名卡.png
已到最後一張卡片，停止翻頁
停發卡總數: 11 張，截圖已完成！
